In [ ]:
# Import TopologicPy modules. This is not needed on other computers
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

from topologicpy.Vertex import Vertex
from topologicpy.Face import Face
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Graph import Graph
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from topologicpy.Vector import Vector
from topologicpy.Helper import Helper
print("This jupyter notebook reuires topologicpy v.0.8.10 or newer")
print(Helper.Version())

In [ ]:
# Choose your rendering environment
renderer = "vscode"
# renderer = "notebook"
# renderer = "jupyterlab"
# renderer = "colab"
# renderer = "browser"

v1 = Vertex.ByCoordinates(10, 5, 1.5)
v2 = Vertex.ByCoordinates(5, 15, 1.5)
v3 = Vertex.ByCoordinates(30, 10, 4.5)
v4 = Vertex.ByCoordinates(20, 25, 4.5)
v5 = Vertex.ByCoordinates(11.5, 5, 4.5)
v6 = Vertex.ByCoordinates(5, 15, 4.5)
v7 = Vertex.ByCoordinates(15.1, 15, 4.5)
v8 = Vertex.ByCoordinates(11.8, 5, 8)

c1 = Cell.Prism(origin=v1, width=19.8, length=9.8, height=2.8)
c2 = Cell.Prism(origin=v2, width=9.8, length=9.8, height=2.8)
c3 = Cell.Prism(origin=v3, width=19.8, length=19.8, height=8.2)
extension = Cell.Prism(origin=v7, width=10, length=9.8, height=8.2)
c3 = Topology.Union(c3, extension)
c3 = Topology.RemoveCoplanarFaces(c3)
c4 = Cell.Wedge(origin=v4, width=39.8, length=9.8, height=8.8, flipHorizontal=True)
c5 = Cell.Prism(origin=v5, width=16, length=9.8, height=2.6)
c6 = Cell.Prism(origin=v6, width=9.8, length=9.8, height=2.8)
c7 = Cell.Wedge(origin=v8, width=16, length=9.8, height=4, flipHorizontal=True)

spaces = [c1, c2, c3, c4, c5, c6, c7]
Topology.Show(spaces, renderer=renderer)


In [ ]:
# An alternative method to get the grid values automatically:

x_epsilon = 0.81
y_epsilon = 0.51
z_epsilon = 0.51
mantissa = 2

all_vertices = [v for space in spaces for v in Topology.Vertices(space)]
x_vertices, y_vertices, z_vertices = zip(*[[Vertex.X(v, mantissa=mantissa), 
                                            Vertex.Y(v, mantissa=mantissa), 
                                            Vertex.Z(v, mantissa=mantissa)] for v in all_vertices])

x_list = Helper.BinAndAverage(list(x_vertices), tolerance=x_epsilon, mantissa=1)
y_list = Helper.BinAndAverage(list(y_vertices), tolerance=y_epsilon, mantissa=1)
z_list = Helper.BinAndAverage(list(z_vertices), tolerance=z_epsilon, mantissa=1)

# Or if you wish to manually set the grids
# x_list = [0, 3.5, 10, 20, 40]
# y_list = [0, 10, 20, 30]
# z_list = [0, 3, 6, 9, 10]

print("X Axis:", x_list)
print("Y Axis:", y_list)
print("Z Axis:", z_list)



In [ ]:

new_spaces = []
for space in spaces:
    vertices = Topology.Vertices(space)
    new_vertices = []
    for v in vertices:
        new_v = Vertex.AlignCoordinates(v, xList=x_list, yList=y_list, zList=z_list, xEpsilon=x_epsilon, yEpsilon=y_epsilon, zEpsilon= z_epsilon)
        new_vertices.append(new_v)
    new_space = Topology.SelfMerge(Topology.ReplaceVertices(space, verticesA = vertices, verticesB = new_vertices))
    new_spaces.append(new_space)

cellComplex = CellComplex.ByCells(new_spaces)
g = Graph.ByTopology(cellComplex, direct=False, viaSharedTopologies=True)
Topology.Show(cellComplex, g, renderer=renderer)

In [ ]:
cells = Topology.Cells(cellComplex)
visited = []
for i, cell_a in enumerate(cells):
    for j, cell_b in enumerate(cells):
        if i != j and not (i,j) in visited:
            edges_a = Topology.Edges(cell_a)
            edges_b = Topology.Edges(cell_b)
            total_area = 0
            dict = Topology.SharedTopologies(cell_a, cell_b)
            shared_faces = dict['faces']
            if len(shared_faces) > 0:
                for shared_face in shared_faces:
                    total_area += Face.Area(shared_face)
                
                #Topology.Show(shared_faces,edges_a, edges_b, faceColor="red", renderer=renderer)
                print("  Cells",i+1, j+1,"share", total_area,"sq.m.")
            else:
                print("X Cells",i+1, j+1,"do not share any faces.")
                #Topology.Show(edges_a, edges_b, renderer=renderer)

            visited.append((i,j))
            visited.append((j,i))